In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
train_dataset = np.load('train_dataset_10k_60.npz')
val_dataset = np.load('validation_dataset_10k_40.npz')
test_dataset = np.load('test_dataset.npz')

# =========== Loading Datasets ===============


x_train = train_dataset['x'].reshape(6000, 784).astype("float32") / 255
y_train = train_dataset['y'].astype("float32")
  
x_val = val_dataset['x'].reshape(4000, 784).astype("float32") / 255
y_val = val_dataset['y'].astype("float32")   
                    
x_test = test_dataset['x'].reshape(10010, 784).astype("float32") / 255
y_test = test_dataset['y'].astype("float32")                    


x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape 

((6000, 784), (6000,), (4000, 784), (4000,), (10010, 784), (10010,))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(784,)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)    # didn't use softmax since it will be called when (logits=true) in below step
])
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
@tf.function
def loss(w1,w2,lamda,loss_fn,y_train,logits): # Lambda
    total_loss = loss_fn(y_train,logits)
    return total_loss + tf.math.exp(lamda)*(tf.nn.l2_loss(w1)+tf.nn.l2_loss(w2))/(2*y_train.shape[0])


In [ ]:
wt_layer1_init = model.layers[1].get_weights()
wt_layer2_init = model.layers[2].get_weights()

train_df = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_df = train_df.shuffle(buffer_size = 750).batch(64)

def fmin_loss(lamda, epochs= 50, l_rate= 0.2, momentum= 0.079, t_hold = 1e-6, nesterov = True):      # lamda, not exp(lamda), Works with both tf.Variable and tf.constant type lambda input, (or just scalar)
    tf.keras.backend.clear_session()
    optimizer = keras.optimizers.SGD(learning_rate=l_rate,momentum = momentum , nesterov =nesterov )
    total_loss0 = 1e20

    for epoch in range(epochs):
        for step,(x_train,y_train) in enumerate(train_df):
            with tf.GradientTape() as tape:
                logits = model(x_train, training=True)
                w1 = model.layers[1].weights[0]
                w2 = model.layers[2].weights[0]
                total_loss1 = loss(w1,w2,lamda,loss_fn,y_train,logits)
                
            vars_list = model.trainable_weights
            grads = tape.gradient(total_loss1, vars_list)      # for ref  - https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough 
            optimizer.apply_gradients(zip(grads,vars_list))

        total_loss0 = total_loss1

    wt_layer1 = model.layers[1].get_weights()
    wt_layer2 = model.layers[2].get_weights()
    model.layers[1].set_weights(wt_layer1_init)
    model.layers[2].set_weights(wt_layer2_init)

    return [total_loss1, wt_layer1, wt_layer2]

In [ ]:
def f_val(w1,b1,w2,b2,lamda): # lamda -> Variable
    layer1_weights = [w1,b1]
    layer2_weights = [w2,b2]
    model.layers[1].set_weights(layer1_weights)
    model.layers[2].set_weights(layer2_weights)
    logits = model(x_train)
    return loss(w1,w2,lamda,loss_fn,y_train,logits)

def f_grad(w1,b1,w2,b2,lamda): # lamda -> Variable
    with tf.GradientTape() as tape:
        total_loss = f_val(w1,b1,w2,b2,lamda)
    vars_list = model.trainable_weights
    vars_list.append(lamda)
    grads = tape.gradient(total_loss, vars_list) 
    return grads

def F_val(w1,b1,w2,b2):     
    layer1_weights = [w1,b1]
    layer2_weights = [w2,b2]
    model.layers[1].set_weights(layer1_weights)
    model.layers[2].set_weights(layer2_weights)
    logits = model(x_val)
    return loss_fn(y_val,logits)

def F_grad(w1,b1,w2,b2):   # float32 arrays
    with tf.GradientTape() as tape:
        total_loss = F_val(w1,b1,w2,b2)
    vars_list = model.trainable_weights
    grads = tape.gradient(total_loss, vars_list) 
    return grads

### GPR

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import matplotlib.pyplot as plt

def GPR( Lamda_sample, Phi_sample, lamda ):

    kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e4))
    gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
    gaussian_process.fit(Lamda_sample, Phi_sample)

    k2_l = gaussian_process.kernel_.get_params()['k2__length_scale']
    x = lamda
    y_pred, sigma = gaussian_process.predict(x, return_std=True)
    y_pred_grad = 0.0*y_pred

    for key, x_star in enumerate(x):

        k_val=gaussian_process.kernel_( Lamda_sample , np.atleast_2d(x_star) , eval_gradient=False).ravel() 
        x_diff_over_l_sq = ((Lamda_sample-x_star)/np.power(k2_l,2)).ravel()
        intermediate_result = np.multiply(k_val, x_diff_over_l_sq)
        final_result = np.dot(intermediate_result, gaussian_process.alpha_)
        y_pred_grad[key] = final_result
        
    return [y_pred[0], y_pred_grad[0]]

In [ ]:
def Function_Grad( w1,b1,w2,b2,lamda, Lamda_sample, Phi_sample, R_val, Mu_val ):

    Phi = GPR(Lamda_sample, Phi_sample, np.array([[lamda.numpy()[0]]]))
    Phi_val, Phi_grad = Phi[0], Phi[1]
    
    p1 = F_grad( w1,b1,w2,b2 ) 
    p2 = ( R_val* ( f_val(w1,b1,w2,b2,lamda) -  Phi_val) + Mu_val )
    f_gradient = f_grad(w1,b1,w2,b2,lamda)
    p3 = f_gradient[:-1] + [f_gradient[-1]-Phi_grad]
    p2tp3 = [p2*elt for elt in p3]

    gradients = [e1+e2 for e1,e2 in zip(p1,p2tp3[:-1])] + [p2tp3[-1]]
    gradients = [(tf.clip_by_norm(grad, clip_norm = 2.0)) for grad in gradients]

    return gradients   # Output : w1, b1, w2, b2, lamda : floar32 arrays

In [ ]:
def AugLag_Function( w1,b1,w2,b2,lamda, Lamda_sample, Phi_sample, R_val, Mu_val ):
    PHI = GPR(Lamda_sample, Phi_sample, np.array([[lamda.numpy()[0]]]))
    f_phi = f_val(w1,b1,w2,b2,lamda) - PHI[0]
    final_val = F_val(w1,b1,w2,b2) + (R_val/2) * f_phi**2 + Mu_val * f_phi
    return final_val.numpy()[0]

In [ ]:
def Gradient_Descent( w1,b1,w2,b2,lamda,Lamda_sample, Phi_sample, R_val, Mu_val, momentum, learning_rate, 
                     epochs = 50, Opt_tol = 1e-10, F_tol = 1e-6 ): # With Nesterov Momentum

    w1_update0 = np.zeros((784,100),dtype='float32')
    b1_update0 = np.zeros((100),dtype='float32')
    w2_update0 = np.zeros((100,10),dtype='float32')
    b2_update0 = np.zeros((10),dtype='float32')
    lamda_update0 = np.zeros((1),dtype='float32')

    w1_weight0, w2_weight0, b1_weight0, b2_weight0, lamda_weight0 = w1, w2, b1, b2, lamda  # Initialized weights
    Loss_val = 1e+20
    Z_VAL0 = 1e+20
    Opt_weights = []
    
    for epoch in range(epochs):

        w1_weight_ahead = w1_weight0 - momentum*w1_update0
        b1_weight_ahead = b1_weight0 - momentum*b1_update0
        w2_weight_ahead = w2_weight0 - momentum*w2_update0
        b2_weight_ahead = b2_weight0 - momentum*b2_update0

        lamda_weight_ahead = lamda_weight0 - momentum*lamda_update0
        lamda_weight_ahead = tf.Variable(lamda_weight_ahead.numpy(), dtype = tf.float32)

        Grad_list = Function_Grad( w1_weight_ahead, b1_weight_ahead, w2_weight_ahead, b2_weight_ahead, 
                                  lamda_weight_ahead, Lamda_sample, Phi_sample, R_val, Mu_val )

        learning_rate = tf.constant(learning_rate, dtype=tf.float32)
        momentum = tf.constant(momentum, dtype=tf.float32)

        part1 = [momentum*update for update in [w1_update0, b1_update0, w2_update0, b2_update0, lamda_update0]]
        part2 = [learning_rate * grad for grad in Grad_list]

        [w1_update0, b1_update0, w2_update0, b2_update0, lamda_update0] = [i+j for i,j in zip(part1,part2)]  
        [w1_weight0, b1_weight0, w2_weight0, b2_weight0, lamda_weight0] = np.subtract([w1_weight0, b1_weight0, w2_weight0, b2_weight0, lamda_weight0], 
                                                                                      [w1_update0, b1_update0, w2_update0, b2_update0, lamda_update0])
        
        Z_VAL = AugLag_Function( w1_weight0, b1_weight0, w2_weight0, b2_weight0, lamda_weight0,
                                Lamda_sample, Phi_sample, R_val, Mu_val )
        Optimality_Tolerance = np.array([np.linalg.norm( optgd, ord = np.inf ) for optgd in Grad_list])

        if Z_VAL <= Loss_val :
            
            Opt_weights = [w1_weight0, b1_weight0, w2_weight0, b2_weight0, lamda_weight0]
            fun_tol = abs( (Z_VAL-Loss_val)/(1+abs(Loss_val)) )
            
            if  fun_tol <= F_tol and np.all(Optimality_Tolerance) <= Opt_tol:
                Loss_val = Z_VAL
                break
                
            Loss_val = Z_VAL
        #=====================================================================================

    return Opt_weights, Loss_val   # w1, b1, w2, b2, lamda


In [ ]:
def LOSS_FUNCTION( wts1, wts2 ):
  
  model.layers[1].set_weights(wts1)
  model.layers[2].set_weights(wts2)

  val_logits = model(x_val)
  val_loss = loss_fn(y_val,val_logits)

  train_logits = model(x_train)
  training_loss = loss_fn(y_train,train_logits)

  test_logits = model(x_test)
  test_loss = loss_fn(y_test,test_logits)

  return val_loss, training_loss, test_loss

In [ ]:
# GLOBAL : Lamda_sample, Phi_sample
def Augmented_Lagrangian( w1,b1,w2,b2,lamda, Lamda_sample, Phi_sample, mom, lr, R_val = 2,
                         Mu_val = 2, neta = 1.5, al_epochs = 5 ):
    Z_val0 = 1e+20
    wts_opt0, lamda_opt0, min_violation = [], 0, 10
    for epoch in range(al_epochs):

        Opt_Weights, _ = Gradient_Descent( w1,b1,w2,b2,lamda, Lamda_sample, Phi_sample, R_val, Mu_val, mom, lr)
        
        w1_opt,b1_opt,w2_opt,b2_opt,lamda_opt = Opt_Weights[0], Opt_Weights[1], Opt_Weights[2], Opt_Weights[3], Opt_Weights[4]  # lamda not var
        
        valL, trL, testL = LOSS_FUNCTION( [w1_opt,b1_opt], [w2_opt,b2_opt] )

        # ========== Setting Weights for Next Loop ====================
        #w1,b1,w2,b2,lamda = w1_opt,b1_opt,w2_opt,b2_opt,lamda_opt
        
        # ============== Updating Augmented Lagrangian Parameters ====================
        Phi1 = GPR(Lamda_sample, Phi_sample, np.array([[lamda_opt.numpy()[0]]]))
        Phi_val1 = Phi1[0]
        lamda_opt = tf.Variable(lamda_opt, dtype=tf.float32)                   # Converting lamda to var
        constraint_ = f_val(w1_opt,b1_opt,w2_opt,b2_opt,lamda_opt) - Phi_val1
        Mu_val = Mu_val + R_val*( constraint_ )
        R_val  = neta*R_val

        # ===== Constraint Violation Criteria ===================
        violation = abs(constraint_)
        print("\n\n Violation = ", violation, " Lambda = ", lamda_opt.numpy()[0], f" Train_Loss : {trL}, Val_Loss : {valL}, Test_Loss : {testL}")
        if violation <= min_violation:
          wts_opt0 = [ [w1_opt, b1_opt], [w2_opt, b2_opt] ]
          lamda_opt0 = lamda_opt
          min_violation = violation
        
        # ==========UPDATING LAMBDA SAMPLE FOR GPR=========================
        Lamda_sample = np.sort(np.array(np.vstack([Lamda_sample,[lamda_opt.numpy()]]), dtype = 'float32'), axis = 0)
        index = np.where(Lamda_sample == [lamda_opt.numpy()[0]] )
        Phi_new_lamda = fmin_loss(lamda_opt)[0].numpy()[0]
        Phi_sample = np.insert(Phi_sample, index[0][0], Phi_new_lamda)
      
    return wts_opt0, lamda_opt0 #, min_violation

In [ ]:
Lamda_sample = np.arange(-10,0,1, dtype = 'float32').reshape((-1,1))

Phi_sample, Weights, lamda_init = [], [], 0
init_val_loss = 1e20
for lamda in Lamda_sample:
    
    [min_loss, layer1wt, layer2wt] = fmin_loss(lamda)
    Phi_sample.append( min_loss.numpy()[0] )
    
    model.layers[1].set_weights(layer1wt)
    model.layers[2].set_weights(layer2wt)
    val_logits = model(x_val)
    val_loss = loss_fn(y_val,val_logits)
    if val_loss <= init_val_loss :
        init_val_loss = val_loss
        Weights = [layer1wt, layer2wt]
        lamda_init = lamda
        
Phi_sample = np.array(Phi_sample)
print("\n\n Phi_sample: ", Phi_sample, "\n\n")

#============================== Initializing Weights =================================================
# w1, b1 = Weights[0][0], Weights[0][1]
# w2, b2 = Weights[1][0], Weights[1][1]
# lamda = tf.Variable(lamda_init, dtype = tf.float32)




 Phi_sample:  [0.00377333 0.00186253 0.00201149 0.00309655 0.00741266 0.0183509
 0.03982027 0.07663073 0.13814265 0.31707436] 




In [ ]:
learn_rate = np.array( [0.001, 0.01, 0.05, 0.07, 0.1], dtype = 'float32' )
momentum_set = np.array( [0.1, 0.5, 0.8, 0.9, 0.99] )

opt_val = 1e20
l0,m0 = 0,0
for lr in learn_rate:
    for mom in momentum_set:
        w1, b1 = Weights[0][0], Weights[0][1]
        w2, b2 = Weights[1][0], Weights[1][1]
        lamda = tf.Variable(lamda_init, dtype = tf.float32)

        wt, val = Gradient_Descent( w1,b1,w2,b2,lamda,Lamda_sample, Phi_sample, R_val = 2, Mu_val = 2,
                                   momentum = mom, learning_rate = lr )
        print("\n LR : {}, MOM : {}, Z_VAL : {}".format(lr,mom,val))
        
        if val <= opt_val:
            opt_val = val
            l0,m0 = lr,mom
print("\n\n Optimum : ",  opt_val, " Momentum : {}, LR : {}".format(m0,l0))


 LR : 0.0010000000474974513, MOM : 0.1, Z_VAL : -0.07093600928783417

 LR : 0.0010000000474974513, MOM : 0.5, Z_VAL : -0.0761214941740036

 LR : 0.0010000000474974513, MOM : 0.8, Z_VAL : -0.09148558974266052

 LR : 0.0010000000474974513, MOM : 0.9, Z_VAL : -0.11584392189979553

 LR : 0.0010000000474974513, MOM : 0.99, Z_VAL : -0.3224531412124634

 LR : 0.009999999776482582, MOM : 0.1, Z_VAL : -0.12931270897388458

 LR : 0.009999999776482582, MOM : 0.5, Z_VAL : -0.19685828685760498

 LR : 0.009999999776482582, MOM : 0.8, Z_VAL : -0.4801451563835144

 LR : 0.009999999776482582, MOM : 0.9, Z_VAL : -0.6305465698242188

 LR : 0.009999999776482582, MOM : 0.99, Z_VAL : -0.6744570732116699

 LR : 0.05000000074505806, MOM : 0.1, Z_VAL : -0.5350652933120728

 LR : 0.05000000074505806, MOM : 0.5, Z_VAL : -0.6309939026832581

 LR : 0.05000000074505806, MOM : 0.8, Z_VAL : -0.66331946849823

 LR : 0.05000000074505806, MOM : 0.9, Z_VAL : -0.688239336013794

 LR : 0.05000000074505806, MOM : 0.99, Z_V

## LLO = 10(+5)

###### Note : The result might differ a little because of GPR estimation every time the program is executed. Training the program to its optimal is avoided because it fits the model over training and validation dataset too well, leading to overfitting on validation dataset. 

In [ ]:
# ===========FINDING LOSS=======================

w1, b1 = Weights[0][0], Weights[0][1]
w2, b2 = Weights[1][0], Weights[1][1]
lamda = tf.Variable(lamda_init, dtype = tf.float32)

wt_final, lamda_final = Augmented_Lagrangian( w1, b1, w2, b2, lamda, Lamda_sample, Phi_sample, m0, l0 )

Final_weights_1, Final_weights_2 = wt_final[0], wt_final[1]
val_loss, training_loss, test_loss = LOSS_FUNCTION(Final_weights_1,Final_weights_2 )

print(f'Validation Loss : {val_loss}, Training Loss : {training_loss}, Testing Loss : {test_loss}')



 Violation =  tf.Tensor([0.49440676], shape=(1,), dtype=float32)  Lambda =  0.2892324  Train_Loss : 0.016379257664084435, Val_Loss : 0.019611740484833717, Test_Loss : 0.16909007728099823


 Violation =  tf.Tensor([0.40509504], shape=(1,), dtype=float32)  Lambda =  0.22615786  Train_Loss : 0.1483832448720932, Val_Loss : 0.04130925238132477, Test_Loss : 0.2407130002975464


 Violation =  tf.Tensor([0.06619643], shape=(1,), dtype=float32)  Lambda =  -4.9222274  Train_Loss : 0.08554207533597946, Val_Loss : 0.03207075968384743, Test_Loss : 0.1961236447095871


 Violation =  tf.Tensor([0.03391527], shape=(1,), dtype=float32)  Lambda =  -4.160192  Train_Loss : 0.05694304034113884, Val_Loss : 0.02920548990368843, Test_Loss : 0.18136821687221527


 Violation =  tf.Tensor([0.00270282], shape=(1,), dtype=float32)  Lambda =  -3.9523466  Train_Loss : 0.046752117574214935, Val_Loss : 0.028081873431801796, Test_Loss : 0.17654967308044434
Validation Loss : 0.028081873431801796, Training Loss : 0.046

In [ ]:
# ===========FINDING LOSS=======================

w1, b1 = Weights[0][0], Weights[0][1]
w2, b2 = Weights[1][0], Weights[1][1]
lamda = tf.Variable(lamda_init, dtype = tf.float32)

wt_final, lamda_final = Augmented_Lagrangian( w1, b1, w2, b2, lamda, Lamda_sample, Phi_sample, m0, l0 )

Final_weights_1, Final_weights_2 = wt_final[0], wt_final[1]
val_loss, training_loss, test_loss = LOSS_FUNCTION(Final_weights_1,Final_weights_2 )

print(f'Validation Loss : {val_loss}, Training Loss : {training_loss}, Testing Loss : {test_loss}')



 Violation =  tf.Tensor([0.49439812], shape=(1,), dtype=float32)  Lambda =  0.28912997  Train_Loss : 0.016379427164793015, Val_Loss : 0.019611859694123268, Test_Loss : 0.16908898949623108


 Violation =  tf.Tensor([0.29424903], shape=(1,), dtype=float32)  Lambda =  1.3477703  Train_Loss : 0.1025828942656517, Val_Loss : 0.03368252515792847, Test_Loss : 0.2100342959165573


 Violation =  tf.Tensor([0.00484691], shape=(1,), dtype=float32)  Lambda =  -3.5372188  Train_Loss : 0.06342198699712753, Val_Loss : 0.028537597507238388, Test_Loss : 0.1852010041475296


 Violation =  tf.Tensor([0.01933116], shape=(1,), dtype=float32)  Lambda =  -3.5505824  Train_Loss : 0.060265492647886276, Val_Loss : 0.030327534303069115, Test_Loss : 0.18351085484027863


 Violation =  tf.Tensor([0.02214812], shape=(1,), dtype=float32)  Lambda =  -1.7875507  Train_Loss : 0.09944881498813629, Val_Loss : 0.033670440316200256, Test_Loss : 0.2031596302986145
Validation Loss : 0.028537597507238388, Training Loss : 0.0